## Text Network

### Install dependencies

In [413]:
import re
import itertools
import spacy
from nltk import FreqDist, bigrams, pos_tag, word_tokenize
import numpy as np
import pandas as pd
import networkx as nx
from pyvis.network import Network
import seaborn as sns
from collections import Counter
sns.set(rc={'figure.figsize':(15,7)})


from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from youtube_transcript_api import YouTubeTranscriptApi
from gensim.models import KeyedVectors

is_noun = lambda word: pos_tag(word_tokenize(word))[0][1] == 'NN'

In [399]:
from nltk import download as nltkdownload
nltkdownload('punkt')
nltkdownload('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [432]:
pos_tag(["another"])

[('another', 'DT')]

In [433]:
def generate_transcript(id):
	transcript = YouTubeTranscriptApi.get_transcript(id,languages=['en'])
	script = ""

	for text in transcript:
		t = text["text"]
		if t != '[Music]':
			script += t + " "
		
	return script, len(script.split())

iden = 'AH15-efqkzY'

#ID é a parte final do link, depois do "=", exemplo 
#https://www.youtube.com/watch?v=L_jWHffIx5E
#L_jWHffIx5E

transcript, no_of_words = generate_transcript(iden)
print(transcript,no_of_words)

but there's one nation with a story that's bigger than them all [Applause] china in china everything is about how do you accumulate more wealth it's a brutal system if you win you win big if you lose you lose everything the great wall of china sends a statement to foreigners it says this is what we can do not just to you but to ourselves what is it about china that makes this such a land of extremes it's a tale of war danger is anywhere and everywhere by the time the war was over china was essentially on its knees one thing that that could be sold to the chinese was opium it's a tale of adversity only in china can motherhood be not a joy but a crime they will break the bone and then bend your bones inward it's a tale of success at any cost chairman now is the red son in our hearts he once said it didn't matter to him if millions of people died if he got the utopia that he was hoping for you have an enormous number of teachers in high schools being beaten to death by their pupils but ho

In [434]:
transcript2 = YouTubeTranscriptApi.get_transcript(iden,languages=['en'])

# dicionário do momento de cada fala
legend_dict = {}
for text in transcript2:
    legend_dict[text['start']] = text['text']
    


In [25]:

modelo = KeyedVectors.load_word2vec_format("C:/Users/rafae/Downloads/MindTube-main (2)/MindTube-main/notebook/wiki-news-300d-1M.vec")

KeyboardInterrupt: 

### Auxiliary functions

In [435]:
toktok = ToktokTokenizer()
stpwrd = stopwords.words('english')
stop_list = (['uh', 'also', 'oh', 'um', 'yeah', 'use', 'lot', 'put', 'get', 'would', 'gonna', 'really', 'much', 'actually', 'another'])
stpwrd.extend(stop_list)

'''for word in stpwrd:
    print(word)
    '''
    
    
#Acha as N palavras mais similares a string passada e retorna uma lista com as palavras. (Sem a métrica de proximidade)
def getSimilar(string, N):
    similarList = modelo.most_similar(positive=[string])[0:N]
    wordList = []
    for word in similarList:
        wordList.append(word)
    return wordList

def sort_list(list1, list2):
     
    list3 = []
    for entry in list2:
        list3.append(entry[1])
    zipped_pairs = zip(list3, list1)
 
    z = [x for _, x in sorted(zipped_pairs, reverse = True)]
 
    return z



#Busca na label dos nós por palavras similares a stopwords
def banStopword(lista, n):
    SUSwords = []
    similarWords = []
    for word in lista:
        similares = getSimilar(word,n)
        mostSimilar = 0
        similarWord = ''
        SUSWord = ''
        for palavra in similares:
            sentinel = 0
            if palavra[0] in stpwrd and palavra[1] > mostSimilar:
                mostSimilar = palavra[1]
                similarWord = palavra
                SUSWord = word
                sentinel = 1
                print(mostSimilar, similarWord, SUSWord)
            if sentinel == 1:  
                SUSwords.append(word)
                similarWords.append(palavra)
    
    return SUSwords, similarWords

def ejectStopword(string, n):
    State = False
    try:
        similares = getSimilar(string.lower(),n)
        for palavra in similares:
            if palavra[0] in stpwrd:
                    State = True
    except KeyError:
        pass
    
    return State

'''word_tokens = toktok.tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

filtered_sentence = []

for w in word_tokens:
	if w not in stop_words:
		filtered_sentence.append(w)

print(word_tokens)
print(filtered_sentence)'''


def filtered_sentence(example_sent):
    word_tokens = toktok.tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stpwrd]
    return " ".join(filtered_sentence)

def remove_stopwords(text):
    doc = filtered_sentence(text)
    print(" ".join([token.text for token in doc if token.is_stop == False]))
    return " ".join([token.text for token in doc if token.is_stop == False])

def pre_processing(text):
    text = filtered_sentence(text)
    text = re.sub(r'[^\w\s]', '', text) #remove punctuation
    text = re.sub(r"\s{2,}", " ", text)
    return text

def co_occurrence(text):
    corpus = list(itertools.chain.from_iterable([text.split(' ')]))
    vocab = list(set(corpus))
    vocab_index = {word: i for i, word in enumerate(vocab)}
    bi_grams = list(bigrams(corpus))
    bigram_freq = FreqDist(bi_grams).most_common(len(bi_grams)) 
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab))) 
    
    #bigram = ((word n, word n+1), num_occurrences)
    for bigram in bigram_freq:
        current = bigram[0][1] #word n+1
        previous = bigram[0][0] #word n
        count = bigram[1] #num_occurrences
        pos_current = vocab_index[current] #obtain id for word n+1 
        pos_previous = vocab_index[previous] #obtain id for word n 
        co_occurrence_matrix[pos_current][pos_previous] = count
        
    return co_occurrence_matrix, vocab_index

def structure_text_network(matrix, vocab_index):
    data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
    data_stack = data_matrix.stack()
    structure = data_stack[data_stack >= 1].rename_axis(('source', 'target')).reset_index(name='weight')
    return structure[(structure.source != " ") & (structure.target != " ")]

In [436]:
lista = ['one', 'measurements', 'what', 'also', 'me', 'mice', 'five', 'standard', 'stat', 'imagine', 'spread', 'weighed', 'look', 'mean', 'another', 'quest', 'quantifies', 'often', 'let', 'plot', 'set', 'deviation', 'data', 'measured', 'differences']


for word in lista:
    state = ejectStopword(word,5)
    print(state)

True
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False


### Generate text network

In [437]:
from spacy.lang.pt.stop_words import STOP_WORDS
nlp = spacy.load("pt_core_news_sm")
#STOP_WORDS.add('gotta')

In [438]:
text = transcript

In [439]:
text_cleaned = pre_processing(text)
matrix, vocab = co_occurrence(text_cleaned)
structure_network = structure_text_network(matrix, vocab)

v_title = "History Of China: 1000 Years of History - History Documentary"

title_words = pre_processing(v_title).split()
title_words

['History', 'China', '1000', 'Years', 'History', 'History', 'Documentary']

In [440]:
structure_network

,source,target,weight
0,,Applause,1.0
1,condemned,time,1.0
2,social,censored,1.0
3,social,birth,1.0
4,social,period,1.0
...,...,...,...
5849,told,battle,1.0
5850,told,factory,1.0
5851,told,century,1.0
5852,tax,kind,1.0


### PageRank

In [441]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])
for i in text_network.nodes:
    for j in text_network.nodes:
        if i != j and i.lower() == j.lower():
            text_network = nx.contracted_nodes(text_network,i, j)

In [442]:
def calc_centrality(G):
    centrality =  nx.pagerank(G)
    
    def mean_simi(word,title):
        m = 0
        for i in title:
            try:
                m += modelo.similarity(word, i)
            except KeyError:
                m += 0

        return m/len(title)

    centrality = {x[0]:(x[1]**0.5)*
                  mean_simi(x[0],title_words)/(mean_simi(x[0],stpwrd)+0.000001)
                  for x in centrality.items()}

    return sorted(centrality.items(), key=lambda x: x[1], reverse=False)

In [443]:
centrality = calc_centrality(text_network)


#dicionário da centralidade
cen_dict = {}
max_cen = 0
for x in centrality:
    cen_dict[x[0]] = x[1]
    if max_cen < x[1]:
        max_cen = x[1]

### Graph Visualization

In [444]:
# lista dos nós
name_lst = [x[0] for x in centrality]

# lista de nós a serem removidos, estamos mantendo os 25 maiores
rmv_lst = name_lst[:-40]

# exibição do nome dos nós para a visualização
for i in text_network.nodes:
    text_network.nodes[i]['label'] = i

# redução do grafo
for i in rmv_lst:
    text_network.remove_node(i)
    
name_lst = list(text_network.nodes)

# remoção de stopwords
for i in name_lst:
    if ejectStopword(i,5) or not is_noun(i):
        print(i)
        merge_node = False
        for j in text_network.neighbors(i):
            w = name_lst.index(j)
            if maxi < w and (not (ejectStopword(j,5) or not is_noun(i))):
                maxi = w
                merge_node = j
            for j in text_network.neighbors(i):
                if merge_node:
                    if not (j == merge_node or (j in text_network.neighbors(merge_node))):
                        text_network.add_edge(merge_node, j, weight=text_network[i][j]['weight']/2)
        text_network.remove_node(i)

know
people
like
years
chinese
one
today
british
great
countries


In [445]:
# momentos em que cada palavra é dita
dict_moments = {}
episilon = 45

for i in text_network.nodes:
    dict_moments[i] = []
    
for i in list(legend_dict.keys()):
    phrase = pre_processing(legend_dict[i])
    
    for j in text_network.nodes:
        if j in phrase.split() and (len(dict_moments[j]) == 0 or  dict_moments[j][-1]+episilon<i):
            dict_moments[j].append((i>1)*(int(i)-1))
            print(f"{j} was appended at {i}")       

Applause was appended at 7.48
china was appended at 11.36
war was appended at 37.68
opium was appended at 49.36
chairman was appended at 66.479
china was appended at 85.439
country was appended at 90.799
communist was appended at 90.799
world was appended at 92.4
xi was appended at 98.0
life was appended at 101.439
century was appended at 102.24
history was appended at 114.64
population was appended at 131.84
world was appended at 139.599
china was appended at 151.44
history was appended at 162.08
shanghai was appended at 178.8
world was appended at 187.76
china was appended at 196.879
century was appended at 201.84
china was appended at 242.799
country was appended at 245.439
city was appended at 247.599
culture was appended at 249.36
world was appended at 254.72
population was appended at 260.32
emperor was appended at 273.68
china was appended at 300.479
population was appended at 306.8
world was appended at 319.52
emperor was appended at 348.32
china was appended at 358.88
populati

In [446]:
nt = Network('800px', '1000px', bgcolor='#222222', notebook=True)
nt.from_nx(text_network)
nt.force_atlas_2based(gravity = -200)
for edg in nt.edges:
    # espessura e física das arestas
    #edg['width'] = text_network[i][j]['weight']**4
    #if edg['weight'] < 1:
    
    if edg['from'] == edg['to']:
        edg['hidden'] = True
    edg['physics'] = True
    edg['color'] = '#cf888f'

for n in nt.nodes:
    # tamanho dos nós
    n['shape'] = 'ellipse'
    size = 40*(cen_dict[n['label']]/max_cen)**0.4
    n['font'] = str(int(size))+'px arial white'
    n['color'] = '#7f333f'
    n['labelHighlightBold'] = True
    

def time_to_min(time):
    return str(time//60)+":"+"0"*(time%60<10)+str(time%60)

for n in nt.nodes:
    # links nos nós
    link_list = ["<a href='https://www.youtube.com/watch?v="+str(iden)+
                 "&t="+str(time)+"s' target='_blank' rel='noopener noreferrer'>"+time_to_min(time)+"<br>" for time in dict_moments[n['label']]]
    soma = ''
    for k in link_list:
        soma += k
    n['title'] = soma
    #print(n['size'])

    
nt.show("text_network.html")

In [218]:
#print(nx.adjacency_matrix(text_network))
#print(nt.nodes)
lista = []
for node in nt.nodes:
    lista.append(node['label'])
print(lista)

['spinning', 'power', 'even', 'time', 'women', 'people', 'England', 'life', 'work', 'Industrial', 'could', 'Many', 'water', 'Like', 'English', 'revolution', 'Production', 'workers', 'One', 'European', 'Europe', 'porcelain', 'industrialization', 'Factories', 'industry']


In [174]:
SUS = banStopword(lista)
print(SUS)

0.6883577704429626 ('doing', 0.6883577704429626) working
0.7955270409584045 ('most', 0.7955270409584045) many
0.6570900082588196 ('get', 0.6570900082588196) see
0.8800161480903625 ('can', 0.8800161480903625) could
0.7335886359214783 ('another', 0.7335886359214783) one
(['working', 'many', 'see', 'could', 'one'], [('doing', 0.6883577704429626), ('most', 0.7955270409584045), ('get', 0.6570900082588196), ('can', 0.8800161480903625), ('another', 0.7335886359214783)])
